In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import time

from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from torch import nn
from train import acc_metric, dice_metric
from DatasetMedical import DatasetMedical
from Unet2D import Unet2D
from train import get_random_folder_split

In [2]:
def test(model, test_dl, acc_fn, dice_fn):
    model.cuda()
    model.eval()
    running_acc = 0
    running_dice = 0
    for x, y in test_dl:
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            outputs = model(x)
            acc = acc_fn(outputs, y)
            dice = dice_fn(outputs, y)
        running_acc += acc
        running_dice += dice
    
    return running_acc/len(test_dl), running_dice/len(test_dl)

In [3]:
def main(params_path):
    bs=1
    
    # load the test data
    base_path = Path('/work/datasets/medical_project/CAMUS_resized')
    _, _, test_files = get_random_folder_split(base_path)
    test_dataset = DatasetMedical(base_path / 'train_gray', test_files,
                                    base_path / 'train_gt')

    # split the training dataset and initialize the data loaders
    test_dl = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    print(len(test_dl))
    xb, yb = next(iter(test_dl))
    print(xb.shape, yb.shape)

    # build the Unet2D with one channel as input and 2 channels as output
    unet = Unet2D(1, 2)
    # Load best model params
    unet.load_state_dict(torch.load(params_path))
    
    acc, dice = test(unet, test_dl, acc_metric, dice_metric)
    print('acc: ', acc,'dice: ', dice)

In [4]:
params_path = "models/model_epoch_42.pth"
main(params_path)

68
torch.Size([1, 1, 384, 384]) torch.Size([1, 384, 384])
acc:  tensor(0.9651, device='cuda:0') dice:  tensor(0.7783, device='cuda:0')
